# Week 4 Tutorial (Exercises) - Latent Variable Models and Autoencoders

### Author: Rajit Rajpal

The goal of this tutorial will be to train various autoencoders (Plain, VAE, IW-VAE) on various datasets (Syntethic, MNIST). `# TODO` are for you to implement. Solutions will contain the completed code.

# 1. Setup
We begin by defining the core components. As shown in the slides, an autoencoder trains a neural network $f_{\theta, \phi}$ to predict $x$ from itself through a bottleneck $z$.

## 1.1 Imports

In [ ]:
# Imports
# %pip install numpy matplotlib torch torchvision scipy pillow scikit-learn <- Uncomment and run this if you do not have the necessary packages.
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import numpy as np
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## 1.2 Architecture

We will have our autoencoder $f_{\theta, \phi}$ be a very simple MLP. Note that the autoencoder has two components: Encoder $E_{\phi}$ and Decoder $D_{\theta}$. In order to keep the rest of the code self-contained, we include VAE within the architecture which can be enabled by setting `is_vae=True`.

In [ ]:
class GenerativeModel(torch.nn.Module):
    def __init__(self, input_dim, latent_dim, is_vae=False):
        super().__init__()
        self.is_vae = is_vae
        # Encoder: compresses high-dim data
        self.enc = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, latent_dim if not is_vae else latent_dim * 2)
        )
        # Decoder: reconstructs data from latents
        self.dec = torch.nn.Sequential(
            torch.nn.Linear(latent_dim, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, input_dim)
        )

    def encode(self, x):
        out = self.enc(x)
        if not self.is_vae:
            return out
        mu, logvar = torch.chunk(out, 2, dim=-1)
        return mu, logvar

    def decode(self, z):
        return self.dec(z)

# 2 Loss Functions

## 2.1 Autoencoder

It is quite simple to train an autoencoder. The simplest thing one can do is minimize the objective: $$ L_{\text{AE}}((\theta, \phi); x) = \|x - D_{\theta}(E_{\phi}(x))\|^2.$$ The norm can be L1 (Mean Absolute Error) or L2 (Mean Squared Error). Try them both, see what happens! We are effectively trying to find a compressed "bottleneck" representation $z \in \R^{d_{\text{latent}}}$ of $x \in \R^{d_{\text{data}}}$.

In [ ]:
def loss_ae(x, x_recon, loss_type='mse'):
    """
    Computes the reconstruction loss for a standard Autoencoder.
    """
    if loss_type == 'mse':
        # Default: Mean Squared Error
        return torch.mean(...) # TODO
    elif loss_type == 'mae':
        # Mean Absolute Error
        return torch.mean(...) # TODO: You can use torch.abs()
    else:
        raise ValueError(f"Unknown loss_type: {loss_type}")

## 2.2 Variational Autoencoder (VAE)

A standard AE does not define a known distribution for $z$, making it impossible to sample new data! The VAE solves this by forcing the latent space to follow a prior $p(z)$, typically a Gaussian $N(0,I)$. We will refer to the probability distribution of the Encoder as $q_{\phi}(z|x)$ and Decoder as $p_{\theta}(x|z)$.

Following Lecture 3 Slide 12, it was shown that:
$$
L_{\text{VAE}}((\theta, \phi); x)
=
\underbrace{
\mathbb{E}_{z \sim q_{\phi}(z|x)}\!\left[\|x - D_{\theta}(z)\|^2\right]
}_{\text{reconstruction loss}}
+
\underbrace{
\mathrm{KL}\!\left(q_{\phi}(z|x)\,\|\,p(z)\right)
}_{\text{prior matching loss}}.
$$

The expectation is obviously approximated using Monte-Carlo.

If $q_{\phi}(z \mid x) = \mathcal{N}(\mu, \sigma^2)$, the KL term has the closed-form solution:

$$
\mathrm{KL}\bigl(q_{\phi}(z \mid x)\,\|\,p(z)\bigr)
=
\frac{1}{2}
\sum_{j=1}^{d_{\text{latent}}}
\left(
\sigma_j^2
+
\mu_j^2
-
1
-
\log \sigma_j^2
\right)
$$



In [ ]:
def loss_vae(x, x_recon, mu, logvar):
    """
    Computes the VAE objective (Negative ELBO).
    Based on Lecture 3, Slide 12.
    """
    # 1. Reconstruction Loss (Standard MSE)
    recon_loss = torch.mean(...) # TODO: Should be same as the one in loss_ae
    
    # 2. Prior-matching Loss (KL Divergence for Gaussians)
    # This forces the latent space to follow N(0, I)
    kl_loss = 0.5 * torch.mean(torch.sum(..., dim=1)) # TODO
    
    return recon_loss + kl_loss

### 2.2.1 Reparameterization Trick

To allow backpropagation through stochastic nodes, we use the reparameterization: $$ z = \mu + \sigma \odot \epsilon $$ where $\epsilon \sim \mathcal{N}(0, I)$.

In [ ]:
def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return ... # TODO

## 2.3 Importance-Weighted VAE (IW-VAE)

We will now implement the **Importance-Weighted Autoencoder (IWAE)** loss. This model addresses a key limitation of the standard VAE: the *looseness* of the Evidence Lower Bound (ELBO).

In a standard VAE, we maximize the ELBO for a single data point $x$:

$$
\log p_{\theta}(x)
\ge
\mathbb{E}_{z \sim q_{\phi}(z \mid x)}
\left[
\log
\frac{p_{\theta}(x, z)}{q_{\phi}(z \mid x)}
\right]
$$

The IWAE improves this by using $K$ samples from the approximate posterior
to create a tighter lower bound:

$$
\log p_{\theta}(x)
\ge
\mathbb{E}_{z_1, \ldots, z_K \sim q_{\phi}(z \mid x)}
\left[
\log
\frac{1}{K}
\sum_{k=1}^{K}
\frac{p_{\theta}(x, z_k)}{q_{\phi}(z_k \mid x)}
\right]
$$

### Why use IWAE?

- **Tighter Bound:** As $K \to \infty$, the IWAE bound approaches the true
  log-likelihood $\log p_{\theta}(x)$.

- **Expressive Posteriors:** IWAE effectively increases the expressiveness of
  the variational posterior without changing the model architecture.

- **Better Likelihood Estimation:** It provides a more accurate estimate for
  evaluating new data.


Please refer to this very well-written blog post for more info on IW-VAE (https://borea17.github.io/paper_summaries/iwae/).

In [ ]:
def loss_iwae(x, mu, logvar, decoder, K=5):
    """
    Computes the IWAE objective as a tighter lower bound on log p(x).
    """
    batch_size = x.size(0)
    log_weights = []

    # Sample K times for each input in the batch 
    for _ in range(K):
        # Sample z from the approximate posterior using reparameterize()
        z_k = reparameterize(mu, logvar) # [batch_size, latent_dim]
        x_recon_k = decoder(z_k)
        
        # log p(x|z) - Log-likelihood of the data given the sample
        # Assuming Gaussian reconstruction with fixed variance
        log_p_x_z = -torch.sum((x - x_recon_k)**2, dim=1)
        
        # log p(z) - Log-prior probability of the sample
        log_p_z = -0.5 * torch.sum(z_k**2, dim=1)
        
        # log q(z|x) - Log-probability of the sample under the approximate posterior
        log_q_z_x = -0.5 * torch.sum(..., dim=1) # TODO: Try to remember what the Gaussian density function is.
        
        # Compute the importance weight for this sample 
        # log( p(x,z) / q(z|x) ) = log p(x|z) + log p(z) - log q(z|x)
        log_weights.append(log_p_x_z + log_p_z - log_q_z_x)
        
    # Stack samples to shape [K, batch_size]
    log_w_matrix = torch.stack(log_weights)
    
    # Apply the Log-Sum-Exp trick to average over K samples 
    # We negate because we minimize the negative log-likelihood estimate
    iwae_loss = -(torch.logsumexp(log_w_matrix, dim=0) - torch.log(torch.tensor(K, dtype=torch.float))).mean()
    return iwae_loss

## Putting them together...

In [ ]:
def compute_loss(model_type, x, x_recon, mu=None, logvar=None, decoder=None, K=5, loss_type='mse'):
    """
    Consolidated loss function for AE, VAE, and IWAE.
    """
    if model_type == "AE":
        return loss_ae(x, x_recon, loss_type=loss_type)
    
    elif model_type == "VAE":
        if mu is None or logvar is None:
            raise ValueError("VAE requires mu and logvar for the KL term.")
        return loss_vae(x, x_recon, mu, logvar)
    
    elif model_type == "IWAE":
        if mu is None or logvar is None or decoder is None:
            raise ValueError("IWAE requires mu, logvar, and the decoder network.")
        return loss_iwae(x, mu, logvar, decoder, K=K)    
    
    else:
        raise ValueError(f"Unknown model_type: {model_type}")

# 3 Data Loading: Synthetic High-Dimensional GMM

We will generate a synthetic high-dimensional dataset where the data lies on a manifold that can be compressed. A classic choice for this is a Gaussian Mixture Model (GMM) in high dimensions, which provides distinct "classes" for visualization. We will generate data in $\R^{100}$ clustered around 5 centers. Although the data is 100D, the underlying structure is simple enough for a 2D or 3D latent space to capture.

In [ ]:
def load_synthetic_data(n_samples=5000, input_dim=100, n_classes=5):
    """
    Generates high-dimensional synthetic data with distinct clusters.
    """
    # Create clusters in high-dimensional space
    data, labels = make_blobs(
        n_samples=n_samples, 
        n_features=input_dim, 
        centers=n_classes, 
        cluster_std=1.5, 
        random_state=42
    )
    
    # Normalize data
    data = (data - data.mean()) / data.std()
    
    # Convert to PyTorch tensors
    x_train = torch.FloatTensor(data)
    y_train = torch.LongTensor(labels)
    
    dataset = TensorDataset(x_train, y_train)
    return DataLoader(dataset, batch_size=64, shuffle=True), input_dim

# Initialize data
train_loader, input_dim = load_synthetic_data()
print(f"Loaded {len(train_loader.dataset)} samples with input dimension {input_dim}")

# 4 Training

In [ ]:
def train_model(model, train_loader, model_type='VAE', epochs=20, lr=1e-3, K=5):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    
    for epoch in range(epochs + 1):
        total_loss = 0
        for x, _ in train_loader:
            optimizer.zero_grad()
            
            if model_type == 'AE':
                z = model.encode(x)
                x_recon = model.decode(z)
                loss = compute_loss('AE', x, x_recon)
            
            elif model_type == 'VAE':
                mu, logvar = model.encode(x)
                z = reparameterize(mu, logvar)
                x_recon = model.decode(z)
                loss = compute_loss('VAE', x, x_recon, mu=mu, logvar=logvar)
                
            elif model_type == 'IWAE':
                # IWAE requires mu and logvar to sample internally within loss_iwae
                mu, logvar = model.encode(x)
                # We pass the decoder because IWAE needs to reconstruct K times
                loss = compute_loss('IWAE', x, None, mu=mu, logvar=logvar, 
                                    decoder=model.dec, K=K)
                
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        if epoch % 5 == 0 or epoch == 0:
            print(f"[{model_type}] Epoch {epoch}/{epochs}, Average Loss: {total_loss/len(train_loader):.4f}")

# Setup experiment
latent_dim = 2 

# 1. Train Autoencoder
print("--- Training Autoencoder ---")
ae_model = GenerativeModel(input_dim, latent_dim, is_vae=False)
train_model(ae_model, train_loader, model_type='AE', epochs=20)

print("\n" + "="*30 + "\n")

# 2. Train Variational Autoencoder
print("--- Training Variational Autoencoder ---")
vae_model = GenerativeModel(input_dim, latent_dim, is_vae=True)
train_model(vae_model, train_loader, model_type='VAE', epochs=20)

print("\n" + "="*30 + "\n")

# 3. Train Importance-Weighted Autoencoder
print("--- Training Importance-Weighted Autoencoder ---")
# IWAE uses the same stochastic encoder architecture as VAE
iwae_model = GenerativeModel(input_dim, latent_dim, is_vae=True)
train_model(iwae_model, train_loader, model_type='IWAE', epochs=20, K=5)

### Discussion: 

What do you observe about the losses? Is one worse than the other? Why is the scale of IWAE and VAE different?

# 5 Visualization

In [ ]:
def visualize_three_comparison(ae_model, vae_model, iwae_model, train_loader):
    models = [('AE', ae_model), ('VAE', vae_model), ('IWAE', iwae_model)]
    plt.figure(figsize=(18, 5))
    
    for i, (model_type, model) in enumerate(models):
        model.eval()
        latents, labels = [], []
        with torch.no_grad():
            for x, y in train_loader:
                out = model.encode(x)
                z = out[0] if isinstance(out, tuple) else out # Use mu for VAE/IWAE
                latents.append(z)
                labels.append(y)

        latents = torch.cat(latents, dim=0).numpy()
        labels = torch.cat(labels, dim=0).numpy()

        plt.subplot(1, 3, i + 1)
        scatter = plt.scatter(latents[:, 0], latents[:, 1], c=labels, cmap='tab10', alpha=0.6, s=15)
        plt.title(f'{model_type} Latent Space')
        plt.grid(True, linestyle='--', alpha=0.5)
        #if model_type != 'AE': plt.xlim(-4, 4); plt.ylim(-4, 4)
            
    plt.tight_layout()
    plt.show()

visualize_three_comparison(ae_model, vae_model, iwae_model, train_loader)

### Discusion:

Observe the difference in scales. Why do you think VAE is collapsing so close to 0?

# MNIST

Let us repeat that but with images. We will use the famed (but simple) MNIST dataset. Running the code below, it should load MNIST data and save into the same folder under "data/".

In [ ]:
def load_mnist_data(batch_size=128):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: torch.flatten(x)) # Flatten 28x28 to 784
    ])
    
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    return train_loader, 784

# Initialize MNIST
mnist_loader, mnist_dim = load_mnist_data()
print(f"Loaded MNIST: {len(mnist_loader.dataset)} images, Input Dimension: {mnist_dim}")

def visualize_mnist_samples(loader, n_samples=16):
    data_iter = iter(loader)
    images, labels = next(data_iter)
    
    # Reshape flattened 784 back to (1, 28, 28) for visualization
    images = images.view(-1, 1, 28, 28)
    
    # Create a grid
    grid = torchvision.utils.make_grid(images[:n_samples], nrow=4)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
    plt.axis('off')
    plt.title("MNIST Training Samples")
    plt.show()

visualize_mnist_samples(mnist_loader)

In [ ]:
def visualize_all_reconstructions(models_list, loader, n_samples=10):
    """
    Visualizes original images followed by reconstructions from each model side-by-side.
    """
    for _, model in models_list:
        model.eval()
        
    # Get a batch of samples
    data_iter = iter(loader)
    images, _ = next(data_iter)
    images = images[:n_samples]
    
    # Create a grid for (Original + n_models) rows
    plt.figure(figsize=(n_samples * 1.5, (len(models_list) + 1) * 1.5))
    
    # 1. Plot Original Inputs
    for i in range(n_samples):
        plt.subplot(len(models_list) + 1, n_samples, i + 1)
        # Reshape flattened 784 back to 28x28
        plt.imshow(images[i].view(28, 28).cpu().numpy(), cmap='gray')
        plt.axis('off')
        if i == 0:
            plt.title("Original Data", loc='left', fontsize=12, fontweight='bold')
            
    # 2. Plot Reconstructions for each model row by row
    for m_idx, (name, model) in enumerate(models_list):
        with torch.no_grad():
            # Pass through encoder
            out = model.encode(images)
            # Use mu for probabilistic models (VAE/IWAE) or the direct z for AE
            z = out[0] if isinstance(out, tuple) else out 
            
            # Decode the latent back to image space
            recons = model.decode(z)
            recons = recons.view(-1, 28, 28).cpu().numpy()
            
            for i in range(n_samples):
                plt.subplot(len(models_list) + 1, n_samples, (m_idx + 1) * n_samples + i + 1)
                plt.imshow(recons[i], cmap='gray')
                plt.axis('off')
                if i == 0:
                    plt.title(f"{name} Recon", loc='left', fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.show()

## Train

This will take about 4 minutes total.

In [ ]:
latent_dim = 2

# 1. Train MNIST Autoencoder (AE)
print("--- Training MNIST Autoencoder ---")
ae_mnist = GenerativeModel(mnist_dim, latent_dim, is_vae=False)
train_model(ae_mnist, mnist_loader, model_type='AE', epochs=15)

print("\n" + "="*30 + "\n")

# 2. Train MNIST Variational Autoencoder (VAE)
print("--- Training MNIST Variational Autoencoder ---")
vae_mnist = GenerativeModel(mnist_dim, latent_dim, is_vae=True)
train_model(vae_mnist, mnist_loader, model_type='VAE', epochs=15)

print("\n" + "="*30 + "\n")

# 3. Train MNIST Importance-Weighted Autoencoder (IWAE)
print("--- Training MNIST IWAE ---")
iwae_mnist = GenerativeModel(mnist_dim, latent_dim, is_vae=True)
train_model(iwae_mnist, mnist_loader, model_type='IWAE', epochs=15, K=5)

## Visualize in 2D

In [ ]:
visualize_three_comparison(ae_mnist, vae_mnist, iwae_mnist, mnist_loader)

In [ ]:
models_to_compare = [
    ('AE', ae_mnist),
    ('VAE', vae_mnist),
    ('IWAE', iwae_mnist)
]

# Run visualization on the MNIST loader
visualize_all_reconstructions(models_to_compare, mnist_loader)

### Discussion:

Do we observe the same issue with VAE? How does it translate when reconstructing a new image? What do you observe about the difference between the AE and IWAE latent spaces?

## Increase latent dim

In [ ]:
latent_dim = 3

# 1. Train MNIST Autoencoder (AE)
print("--- Training MNIST Autoencoder ---")
ae_mnist = GenerativeModel(mnist_dim, latent_dim, is_vae=False)
train_model(ae_mnist, mnist_loader, model_type='AE', epochs=15)

print("\n" + "="*30 + "\n")

# 2. Train MNIST Variational Autoencoder (VAE)
print("--- Training MNIST Variational Autoencoder ---")
vae_mnist = GenerativeModel(mnist_dim, latent_dim, is_vae=True)
train_model(vae_mnist, mnist_loader, model_type='VAE', epochs=15)

print("\n" + "="*30 + "\n")

# 3. Train MNIST Importance-Weighted Autoencoder (IWAE)
print("--- Training MNIST IWAE ---")
iwae_mnist = GenerativeModel(mnist_dim, latent_dim, is_vae=True)
train_model(iwae_mnist, mnist_loader, model_type='IWAE', epochs=15, K=5)

### Discussion:

Now that we increased latent dimension, do you notice any differences in the learning? Explain your reasoning.

## Visualize in 3D

In [ ]:
def visualize_three_comparison_3d(ae_model, vae_model, iwae_model, train_loader):
    """
    Plots the 3D latent space of AE, VAE, and IWAE side-by-side.
    """
    models = [('AE', ae_model), ('VAE', vae_model), ('IWAE', iwae_model)]
    fig = plt.figure(figsize=(20, 7))
    
    for i, (model_type, model) in enumerate(models):
        model.eval()
        latents, labels = [], []
        
        with torch.no_grad():
            for x, y in train_loader:
                out = model.encode(x)
                # Use mu for VAE/IWAE, or the direct bottleneck for AE
                z = out[0] if isinstance(out, tuple) else out
                latents.append(z)
                labels.append(y)

        latents = torch.cat(latents, dim=0).numpy()
        labels = torch.cat(labels, dim=0).numpy()

        ax = fig.add_subplot(1, 3, i + 1, projection='3d')
        scatter = ax.scatter(latents[:, 0], latents[:, 1], latents[:, 2], 
                             c=labels, cmap='tab10', alpha=0.5, s=5)
        
        ax.set_title(f'{model_type} 3D Latent Space')
        ax.set_xlabel('z1')
        ax.set_ylabel('z2')
        ax.set_zlabel('z3')
        
        # Consistent scaling for probabilistic models to see the Gaussian prior match
        #if model_type != 'AE':
            #ax.set_xlim(-4, 4); ax.set_ylim(-4, 4); ax.set_zlim(-4, 4)
            
    plt.tight_layout()
    plt.show()

visualize_three_comparison_3d(ae_mnist, vae_mnist, iwae_mnist, mnist_loader)

In [ ]:
models_to_compare = [
    ('AE', ae_mnist),
    ('VAE', vae_mnist),
    ('IWAE', iwae_mnist)
]

# Run visualization on the MNIST loader
visualize_all_reconstructions(models_to_compare, mnist_loader)